Este notebook apunta a realizar los análisis de datos necesarios para responder a la pregunta siguiente: 

- ¿Cuáles son las caractéristicas sociales de las personas que hablan sobre el cambio climático en la prensa en terminos de "edad", "genero", "profesion" y "popularidad"? 

- ¿Cómo evolucionan estas características en el tiempo? 

- ¿Se puede observar tendencias distintas en los paises siguientes: Costa Rica, España, Chile

ver más detalles en el documento borrador: https://docs.google.com/document/d/1LyIBSMz_Suhy7esHSQt49P56asrhoeCb18qbpOggCY8/edit?usp=sharing

# 1- Construcción del corpus de noticias

In [4]:
import pandas as pd
from pandasql import sqldf

In [5]:
file_corpus_espana="cambio_climatico_espana_2018-01-01_2021-12-31.csv"

In [6]:
corpus_espana=pd.read_csv(file_corpus_espana)

In [7]:
corpus_espana

,Unnamed: 0,id_news,country,media_outlet,url,title,text,date,search
0,0,12889490.0,espana,elindependiente,https://www.elindependiente.com/futuro/medio-a...,Los españoles no niegan el cambio climático y ...,Sólo un 10% de los españoles vive al margen de...,2018-12-05,cambio climático
1,1,605350.0,espana,antena3,https://www.antena3.com/noticias/sociedad/deca...,Decálogo para informar correctamente sobre el ...,La Fundación Ecología y Desarrollo (ECODES) y ...,2019-11-18,cambio climático
2,2,306938.0,espana,elespanol,https://www.elespanol.com/espana/politica/2020...,"Borrell y Ribera, 'a la gresca' por su crítica...",El Alto Representante de la UE para Política E...,2020-02-07,cambio climático
3,3,184532.0,espana,antena3,https://www.antena3.com/noticias/espana/teresa...,Teresa Ribera será la ministra de Medio Ambien...,La ex secretaria de Estado de Medio Ambiente y...,2018-06-05,cambio climático
4,4,12865161.0,espana,elindependiente,https://www.elindependiente.com/futuro/medio-a...,"El jefe del IPCC, a los políticos: ""Dígannos q...",El presidente del Grupo Intergubernamental de ...,2019-12-02,cambio climático
...,...,...,...,...,...,...,...,...,...
5128,6283,12758167.0,espana,elperiodico,https://www.elperiodico.com/es/politica/201910...,Así fue la convocatoria de las elecciones gene...,El Periódico Las elecciones generales se tendr...,2019-10-01,cambio climático
5129,6284,12734308.0,espana,elperiodico,https://www.elperiodico.com/es/politica/201910...,Así les hemos contado la actualidad de las pro...,M. Baraza / R. Susín / B. González Catalunya h...,2019-10-30,cambio climático
5130,6285,12924657.0,espana,elperiodico,https://www.elperiodico.com/es/politica/201812...,Los intereses partidistas pueden eternizar la ...,Laura Puig / Julia Camacho El descalabro de Su...,2018-12-03,cambio climático
5131,6286,14299559.0,espana,elperiodico,https://www.elperiodico.com/es/tiempo/20181012...,Así les hemos contado las inundaciones en Mall...,Rosa Mari Sanz / Begoña González Las inundacio...,2018-10-12,cambio climático


- ¿Cuáles son los medios y su número de noticias?

In [9]:
query="""SELECT media_outlet,count(*) as noticias, min(date),max(date)
FROM corpus_espana GROUP BY media_outlet ORDER BY noticias DESC;"""
result=sqldf(query)
result

,media_outlet,noticias,min(date),max(date)
0,elperiodico,1063,2018-01-01,2021-04-26
1,okdiario,809,2018-01-03,2020-12-24
2,elespanol,776,2018-01-02,2020-12-13
3,publico,696,2018-01-01,2020-12-28
4,elindependiente,606,2018-01-01,2021-04-24
5,antena3,605,2018-01-10,2020-12-14
6,larazon,407,2018-09-03,2020-12-27
7,elcorreoespana,84,2018-01-26,2020-12-18
8,cadenaser,49,2018-01-16,2020-05-19
9,abcespana,38,2019-05-10,2020-12-28


- ¿Cómo se distribuyen las noticias en el tiempo?

In [10]:
#Contar el número de documentos por año

query="""SELECT strftime('%Y', date) as year,count(*) as documents
FROM corpus_espana GROUP BY year;"""
result=sqldf(query)
result

,year,documents
0,2018,1013
1,2019,2381
2,2020,1599
3,2021,140


# 2. Detección de las entidades de tipo Personas

In [11]:
from collections import Counter
import spacy
nlp = spacy.load("es_core_news_lg") #cambiar por "es_core_news_lg" para el análisis final

In [12]:
df_persons = pd.DataFrame(columns=["date", "name"])

for index, row in corpus_espana.iterrows():
    print(index)
    text = str(row["text"])
    date = row["date"]
    person_in_text = []
    
    if(len(text)>0):
        doc = nlp(text)
        
        for ent in doc.ents:
            if (ent.label_ == "PER"):
                person = ent.text
                if (' ' in person) == True:
                    if person not in person_in_text:
                        df_persons = df_persons.append({"date": date,"name":  person}, ignore_index=True)
                        person_in_text.append(person)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062


3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701


In [16]:
df_persons

,date,name
0,2020-02-07,Josep Borrell
1,2020-02-07,Greta Thunberg
2,2020-02-07,Teresa Ribera
3,2020-02-07,José Manuel García Margallo
4,2018-06-05,Teresa Ribera
...,...,...
25874,2019-03-08,Teresa Ribera
25875,2019-03-08,Francisco Pardo
25876,2019-03-08,Pilar Allúe
25877,2019-03-08,Eulalia González


In [17]:
query="""SELECT strftime('%Y', date) as year,name, count(*) as freq
FROM df_persons GROUP BY year, name ORDER BY count(*) DESC;"""
df_persons_by_year=sqldf(query)
df_persons_by_year

,year,name,freq
0,2019,Pedro Sánchez,452
1,2019,Greta Thunberg,294
2,2020,Pedro Sánchez,274
3,2018,Pedro Sánchez,204
4,2019,Donald Trump,182
...,...,...,...
13332,2021,Ángel Gurría,1
13333,2021,Ángela Merkel,1
13334,2021,Ángeles Vázquez,1
13335,2021,Ángels Barceló,1


# 3. Extracción de metadatos sobre cada persona (género, popularidad, edad, profesión)

- GENERO

In [18]:
import unidecode

DICT_FIRSTNAMES="names_gender.csv"

gen_nam=pd.read_csv(DICT_FIRSTNAMES)

In [19]:
def get_gender(named_entity, language):
    #We consider the first token  of the named entity as the firstname
    firstname=named_entity.split(' ', maxsplit=1)[0] 
    firstname_upper= firstname.upper()
    firstname_unidecode_upper=unidecode.unidecode(firstname).upper()

    gender_df = gen_nam[(gen_nam.firstname==firstname) | (gen_nam.firstname==firstname_upper) | (gen_nam.firstname==firstname_unidecode_upper)]
    male_df = gender_df[gender_df.gender=='M']
    female_df = gender_df[gender_df.gender=='F']
    if(len(gender_df)!=0):
        if(len(female_df)/len(gender_df) >= 0.6  ):
            return female_df.values[0][2]
        if(len(male_df)/len(gender_df) >= 0.6 ):
            return male_df.values[0][2]
        return 'N'    
    return None

In [20]:
get_gender("Jamal Khashoggi","Spanish")

'M'

In [21]:
df_persons_by_year["gender"] = ""


In [22]:
df_persons_by_year

,year,name,freq,gender
0,2019,Pedro Sánchez,452,
1,2019,Greta Thunberg,294,
2,2020,Pedro Sánchez,274,
3,2018,Pedro Sánchez,204,
4,2019,Donald Trump,182,
...,...,...,...,...
13332,2021,Ángel Gurría,1,
13333,2021,Ángela Merkel,1,
13334,2021,Ángeles Vázquez,1,
13335,2021,Ángels Barceló,1,


In [23]:
for index, row in df_persons_by_year.iterrows():
    print(index)
    gender=get_gender(row["name"],"Spanish")
    df_persons_by_year.loc[index,['gender']] = gender

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066


3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708


5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351


6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994


8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637


10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
10209
10210
10211
10212
10213
10214
10215
10216
10217
10218
10219
10220
10221
10222
10223
10224
10225
10226
10227
10228
10229
10230
10231
10232
10233
1023

11436
11437
11438
11439
11440
11441
11442
11443
11444
11445
11446
11447
11448
11449
11450
11451
11452
11453
11454
11455
11456
11457
11458
11459
11460
11461
11462
11463
11464
11465
11466
11467
11468
11469
11470
11471
11472
11473
11474
11475
11476
11477
11478
11479
11480
11481
11482
11483
11484
11485
11486
11487
11488
11489
11490
11491
11492
11493
11494
11495
11496
11497
11498
11499
11500
11501
11502
11503
11504
11505
11506
11507
11508
11509
11510
11511
11512
11513
11514
11515
11516
11517
11518
11519
11520
11521
11522
11523
11524
11525
11526
11527
11528
11529
11530
11531
11532
11533
11534
11535
11536
11537
11538
11539
11540
11541
11542
11543
11544
11545
11546
11547
11548
11549
11550
11551
11552
11553
11554
11555
11556
11557
11558
11559
11560
11561
11562
11563
11564
11565
11566
11567
11568
11569
11570
11571
11572
11573
11574
11575
11576
11577
11578
11579
11580
11581
11582
11583
11584
11585
11586
11587
11588
11589
11590
11591
11592
11593
11594
11595
11596
11597
11598
11599
11600
11601
1160

12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
12939
12940
12941
12942
12943
12944
12945
12946
12947
12948
12949
12950
12951
12952
12953
12954
12955
12956
12957
12958
12959
12960
12961
12962
12963
12964
12965
12966
12967
12968
12969
1297

In [24]:
df_persons_by_year

,year,name,freq,gender
0,2019,Pedro Sánchez,452,M
1,2019,Greta Thunberg,294,F
2,2020,Pedro Sánchez,274,M
3,2018,Pedro Sánchez,204,M
4,2019,Donald Trump,182,M
...,...,...,...,...
13332,2021,Ángel Gurría,1,M
13333,2021,Ángela Merkel,1,F
13334,2021,Ángeles Vázquez,1,F
13335,2021,Ángels Barceló,1,F


In [25]:
query="""SELECT year, gender, sum(freq), count(*) as nb_persons
FROM df_persons_by_year GROUP BY year, gender ORDER BY year, gender;"""
result=sqldf(query)
result

,year,gender,sum(freq),nb_persons
0,2018,None,446,362
1,2018,F,1198,564
2,2018,M,3272,1691
3,2018,N,24,12
4,2019,None,1268,936
5,2019,F,3251,1328
6,2019,M,7334,3351
7,2019,N,21,18
8,2020,None,946,729
9,2020,F,2168,1079


- POPULARIDAD

In [26]:
import statistics
import pageviewapi

In [27]:
def get_popularity(named_entity, date):
    popularity_en=calculate_popularity(named_entity, date,"en")
    popularity_es=calculate_popularity(named_entity, date,"es")
    popularity_fr=calculate_popularity(named_entity, date,"fr")
    popularity_it=calculate_popularity(named_entity, date,"it")
    popularity_pt=calculate_popularity(named_entity, date,"pt")
    return {"popularity_en":popularity_en,"popularity_es":popularity_es,"popularity_fr":popularity_fr,"popularity_it":popularity_it,"popularity_pt":popularity_pt}

In [28]:
import unidecode
from datetime import timedelta, datetime

def calculate_popularity(named_entity, date, language):
    start_date = (date - timedelta(days=365)).strftime('%Y%m%d')
    end_date = (date).strftime('%Y%m%d')
    try: 
        result=pageviewapi.per_article(f'{language}.wikipedia', named_entity, start_date , end_date,
                        access='all-access', agent='all-agents', granularity='daily')    
    except Exception as error:
        return 0
    views = []
    for item in result.items():
        for article in item[1]:
            views.append(article['views'])
    return int(statistics.mean(views))

In [29]:
get_popularity("Gavin Newsom",datetime.now())

{'popularity_en': 8696,
 'popularity_es': 107,
 'popularity_fr': 91,
 'popularity_it': 51,
 'popularity_pt': 45}

In [30]:
df_persons_by_year["popularity_en"] = ""
df_persons_by_year["popularity_es"] = ""
df_persons_by_year["popularity_fr"] = ""
df_persons_by_year["popularity_it"] = ""
df_persons_by_year["popularity_pt"] = ""

In [31]:
 df_persons_by_year

,year,name,freq,gender,popularity_en,popularity_es,popularity_fr,popularity_it,popularity_pt
0,2019,Pedro Sánchez,452,M,,,,,
1,2019,Greta Thunberg,294,F,,,,,
2,2020,Pedro Sánchez,274,M,,,,,
3,2018,Pedro Sánchez,204,M,,,,,
4,2019,Donald Trump,182,M,,,,,
...,...,...,...,...,...,...,...,...,...
13332,2021,Ángel Gurría,1,M,,,,,
13333,2021,Ángela Merkel,1,F,,,,,
13334,2021,Ángeles Vázquez,1,F,,,,,
13335,2021,Ángels Barceló,1,F,,,,,


- Obtener textos desde Wikipedia, Edad & Profesión

In [32]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import pipeline

In [33]:
ES_MODEL_LANGUAGE="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
EN_MODEL_LANGUAGE="distilbert-base-cased-distilled-squad"

In [34]:
tokenizer_es_language = AutoTokenizer.from_pretrained(ES_MODEL_LANGUAGE)
model_es_language = AutoModelForQuestionAnswering.from_pretrained(ES_MODEL_LANGUAGE)

tokenizer_en_language = AutoTokenizer.from_pretrained(EN_MODEL_LANGUAGE)
model_en_language = AutoModelForQuestionAnswering.from_pretrained(EN_MODEL_LANGUAGE)

In [35]:
q_a_es = pipeline("question-answering", model=model_es_language, tokenizer=tokenizer_es_language)
q_a_en = pipeline("question-answering", model=model_en_language, tokenizer=tokenizer_en_language)

In [36]:
df_persons_by_year["wikipedia"] = ""
df_persons_by_year["birth"] = ""
df_persons_by_year["profession"] = ""

In [37]:
import wikipedia

In [38]:
df_persons_by_year.to_csv("espana_persons_cambioclimatico.csv")

In [39]:
df_persons_by_year = pd.read_csv("espana_persons_cambioclimatico.csv")

In [ ]:
for index, row in df_persons_by_year.iterrows():
        print(index)
        name=row["name"]
        print(name)
        year=row["year"]
        
        ##Popularity
        date = datetime(int(year), 12, 31, 0, 0, 0, 0)
        popularities=get_popularity(name,date)
        print(popularities)
        df_persons_by_year.loc[index,['popularity_en']] = popularities["popularity_en"]
        df_persons_by_year.loc[index,['popularity_es']] = popularities["popularity_es"]
        df_persons_by_year.loc[index,['popularity_fr']] = popularities["popularity_fr"]
        df_persons_by_year.loc[index,['popularity_it']] = popularities["popularity_it"]
        df_persons_by_year.loc[index,['popularity_pt']] = popularities["popularity_pt"]
        
        ##Wikipedia
        lang="en"
        wikipedia.set_lang("en")
        try:
            text = wikipedia.summary(name, sentences=3)
        except:
            wikipedia.set_lang("es")
            lang="es"
            try:
                text = wikipedia.summary(name, sentences=3)
            except:
                print(name+" no está en Wikipedia.")
                
        df_persons_by_year.loc[index,['wikipedia']] = text
        
        ##Edad & Profesión
        if lang == "en":
            result=q_a_en(question="What year was she or he born?", context=text)
            df_persons_by_year.loc[index,['birth']] = result["answer"]
            result=q_a_en(question="What is her or his profession?", context=text)
            df_persons_by_year.loc[index,['profession']] = result["answer"]
        else:
            result = q_a_es(question="¿En qué año nació el o ella?", context=text)
            df_persons_by_year.loc[index,['birth']] = result["answer"]
            result = q_a_es(question="¿Cuál es su profesión?", context=text)
            df_persons_by_year.loc[index,['profession']] = result["answer"]

0
Pedro Sánchez
{'popularity_en': 515, 'popularity_es': 2659, 'popularity_fr': 263, 'popularity_it': 178, 'popularity_pt': 107}
1
Greta Thunberg
{'popularity_en': 22508, 'popularity_es': 5219, 'popularity_fr': 4404, 'popularity_it': 5597, 'popularity_pt': 1768}
2
Pedro Sánchez
{'popularity_en': 1111, 'popularity_es': 3086, 'popularity_fr': 401, 'popularity_it': 209, 'popularity_pt': 103}
3
Pedro Sánchez
{'popularity_en': 30, 'popularity_es': 2894, 'popularity_fr': 241, 'popularity_it': 185, 'popularity_pt': 86}
4
Donald Trump
{'popularity_en': 139852, 'popularity_es': 4312, 'popularity_fr': 3326, 'popularity_it': 1586, 'popularity_pt': 1286}
5
Donald Trump
{'popularity_en': 617453, 'popularity_es': 9681, 'popularity_fr': 7807, 'popularity_it': 4653, 'popularity_pt': 2666}
6
Teresa Ribera
{'popularity_en': 22, 'popularity_es': 93, 'popularity_fr': 5, 'popularity_it': 0, 'popularity_pt': 0}
7
Pablo Iglesias
{'popularity_en': 10, 'popularity_es': 73, 'popularity_fr': 4, 'popularity_it': 2

/home/mvernier/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/mvernier/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Felipe VI no está en Wikipedia.
37
Inés Arrimadas
{'popularity_en': 126, 'popularity_es': 1425, 'popularity_fr': 27, 'popularity_it': 11, 'popularity_pt': 8}
38
Ada Colau
{'popularity_en': 145, 'popularity_es': 592, 'popularity_fr': 83, 'popularity_it': 24, 'popularity_pt': 8}
39
António Guterres
{'popularity_en': 1336, 'popularity_es': 197, 'popularity_fr': 209, 'popularity_it': 102, 'popularity_pt': 194}
40
Carmen Calvo
{'popularity_en': 89, 'popularity_es': 875, 'popularity_fr': 19, 'popularity_it': 15, 'popularity_pt': 0}
41
Charles Michel
{'popularity_en': 655, 'popularity_es': 107, 'popularity_fr': 307, 'popularity_it': 131, 'popularity_pt': 17}
42
Emmanuel Macron
{'popularity_en': 7771, 'popularity_es': 1161, 'popularity_fr': 6400, 'popularity_it': 694, 'popularity_pt': 863}
43
Salvador Illa
{'popularity_en': 130, 'popularity_es': 4381, 'popularity_fr': 16, 'popularity_it': 8, 'popularity_pt': 0}
44
Carmen Calvo
{'popularity_en': 16, 'popularity_es': 726, 'popularity_fr': 16, 'p

102
Isabel Díaz Ayuso
{'popularity_en': 220, 'popularity_es': 3326, 'popularity_fr': 35, 'popularity_it': 25, 'popularity_pt': 0}
103
Carles Puigdemont
{'popularity_en': 1190, 'popularity_es': 929, 'popularity_fr': 562, 'popularity_it': 109, 'popularity_pt': 33}
104
Pedro Duque
{'popularity_en': 87, 'popularity_es': 939, 'popularity_fr': 12, 'popularity_it': 8, 'popularity_pt': 5}
105
Jaume Asens
{'popularity_en': 0, 'popularity_es': 95, 'popularity_fr': 3, 'popularity_it': 0, 'popularity_pt': 0}
106
Juantxo López de Uralde
{'popularity_en': 0, 'popularity_es': 1, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
107
Ada Colau
{'popularity_en': 98, 'popularity_es': 301, 'popularity_fr': 59, 'popularity_it': 24, 'popularity_pt': 6}
108
Bernie Sanders
{'popularity_en': 21439, 'popularity_es': 972, 'popularity_fr': 990, 'popularity_it': 555, 'popularity_pt': 394}
109
Quim Torra
{'popularity_en': 167, 'popularity_es': 712, 'popularity_fr': 43, 'popularity_it': 12, 'popularity_pt'

/home/mvernier/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


...

Cátedra Tarek Ahmed Juffali no está en Wikipedia.
2223
Cédric Villani
{'popularity_en': 292, 'popularity_es': 29, 'popularity_fr': 1177, 'popularity_it': 26, 'popularity_pt': 8}
2224
Daniel Pauly
{'popularity_en': 26, 'popularity_es': 0, 'popularity_fr': 8, 'popularity_it': 0, 'popularity_pt': 1}
2225
David Beasley
{'popularity_en': 335, 'popularity_es': 11, 'popularity_fr': 18, 'popularity_it': 0, 'popularity_pt': 0}
2226
David Belliard
{'popularity_en': 92, 'popularity_es': 0, 'popularity_fr': 55, 'popularity_it': 0, 'popularity_pt': 0}
2227
David Martín Porras
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2228
David Vieites
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2229
Decía Blaise Pascal
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2230
Del Campo
{'popularity_en': 0, 'popularity_es': 1, 'popularity_fr': 0, 'popula

2264
Elon Musk
{'popularity_en': 63840, 'popularity_es': 6267, 'popularity_fr': 4736, 'popularity_it': 3466, 'popularity_pt': 2002}
2265
Elsa Artadi
{'popularity_en': 10, 'popularity_es': 67, 'popularity_fr': 3, 'popularity_it': 0, 'popularity_pt': 0}
2266
Elvira Carles
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
Elvira Carles no está en Wikipedia.
2267
Emiliano García-Page
{'popularity_en': 13, 'popularity_es': 312, 'popularity_fr': 4, 'popularity_it': 2, 'popularity_pt': 0}
2268
Emiliano López Atxurra
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2269
Emilio Santiago
{'popularity_en': 2, 'popularity_es': 2, 'popularity_fr': 1, 'popularity_it': 2, 'popularity_pt': 3}
2270
En la
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2271
Enric Morist
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 

2325
Isabel Díaz-Ayuso
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2326
Isabel Serra
{'popularity_en': 13, 'popularity_es': 221, 'popularity_fr': 3, 'popularity_it': 0, 'popularity_pt': 2}
2327
Isabel Tocino
{'popularity_en': 4, 'popularity_es': 60, 'popularity_fr': 2, 'popularity_it': 1, 'popularity_pt': 0}
2328
Isabelle Guyon
{'popularity_en': 10, 'popularity_es': 0, 'popularity_fr': 5, 'popularity_it': 0, 'popularity_pt': 0}
2329
Isaura Leal
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2330
Isidro Martínez
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2331
Isidro Martínez Oblanca
{'popularity_en': 0, 'popularity_es': 30, 'popularity_fr': 1, 'popularity_it': 0, 'popularity_pt': 0}
2332
Ismael Clemente
{'popularity_en': 0, 'popularity_es': 22, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2333
Ismael Figue

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/mvernier/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_22625/1313760573.py", line 9, in <module>
    popularities=get_popularity(name,date)
  File "/tmp/ipykernel_22625/1800462775.py", line 2, in get_popularity
    popularity_en=calculate_popularity(named_entity, date,"en")
  File "/tmp/ipykernel_22625/3256718859.py", line 8, in calculate_popularity
    result=pageviewapi.per_article(f'{language}.wikipedia', named_entity, start_date , end_date,
  File "/home/mvernier/.local/lib/python3.8/site-packages/pageviewapi/client.py", line 77, in per_article
    return __api__(PA_ENDPOINT, args)
  File "/home/mvernier/.local/lib/python3.8/site-packages/pageviewapi/client.py", line 145, in __api__
    response = requests.get(url, headers=USER_AGENT)
  File "/home/mvernier/.local/lib/python3.8/site-packages/requests/api.py", li

In [40]:
df_persons_by_year

ERROR! Session/line number was not unique in database. History logging moved to new session 217


,Unnamed: 0,year,name,freq,gender,popularity_en,popularity_es,popularity_fr,popularity_it,popularity_pt,wikipedia,birth,profession
0,0,2019,Pedro Sánchez,452,M,515.0,2659.0,263.0,178.0,107.0,Pedro Sánchez Pérez-Castejón (Spanish: [ˈpeðɾo...,1972,politician
1,1,2019,Greta Thunberg,294,F,22508.0,5219.0,4404.0,5597.0,1768.0,Greta Tintin Eleonora Ernman Thunberg (Estocol...,2003,activista medioambiental sueca
2,2,2020,Pedro Sánchez,274,M,1111.0,3086.0,401.0,209.0,103.0,Pedro Sánchez Pérez-Castejón (Spanish: [ˈpeðɾo...,1972,politician
3,3,2018,Pedro Sánchez,204,M,30.0,2894.0,241.0,185.0,86.0,Pedro Sánchez Pérez-Castejón (Spanish: [ˈpeðɾo...,1972,politician
4,4,2019,Donald Trump,182,M,139852.0,4312.0,3326.0,1586.0,1286.0,"Donald John Trump (born June 14, 1946) is an A...",1946,media personality
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13332,13332,2021,Ángel Gurría,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13333,13333,2021,Ángela Merkel,1,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13334,13334,2021,Ángeles Vázquez,1,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13335,13335,2021,Ángels Barceló,1,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df_persons_by_year.to_csv("espana_persons_cambioclimatico_final.csv")

In [42]:
df_persons_by_year = pd.read_csv("espana_persons_cambioclimatico_final.csv")
df_persons_by_year

,Unnamed: 0,Unnamed: 0.1,year,name,freq,gender,popularity_en,popularity_es,popularity_fr,popularity_it,popularity_pt,wikipedia,birth,profession
0,0,0,2019,Pedro Sánchez,452,M,515.0,2659.0,263.0,178.0,107.0,Pedro Sánchez Pérez-Castejón (Spanish: [ˈpeðɾo...,1972,politician
1,1,1,2019,Greta Thunberg,294,F,22508.0,5219.0,4404.0,5597.0,1768.0,Greta Tintin Eleonora Ernman Thunberg (Estocol...,2003,activista medioambiental sueca
2,2,2,2020,Pedro Sánchez,274,M,1111.0,3086.0,401.0,209.0,103.0,Pedro Sánchez Pérez-Castejón (Spanish: [ˈpeðɾo...,1972,politician
3,3,3,2018,Pedro Sánchez,204,M,30.0,2894.0,241.0,185.0,86.0,Pedro Sánchez Pérez-Castejón (Spanish: [ˈpeðɾo...,1972,politician
4,4,4,2019,Donald Trump,182,M,139852.0,4312.0,3326.0,1586.0,1286.0,"Donald John Trump (born June 14, 1946) is an A...",1946,media personality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13332,13332,13332,2021,Ángel Gurría,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13333,13333,13333,2021,Ángela Merkel,1,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13334,13334,13334,2021,Ángeles Vázquez,1,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13335,13335,13335,2021,Ángels Barceló,1,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
